# Load Flow Studies

## Bus Data

In [1]:
import numpy as np

# From To R X
# col0 - From, col1 - To, col2 - R(resistance), col3 - X(reactance)

# Bus Dats

z = np.array([[0, 1, 0, 1],
     [0, 2, 0, 0.8],
     [1, 2, 0, 0.4],
     [1, 3, 0, 0.2],
     [2, 3, 0, 0.2],
     [3, 4, 0, 0.08]])

# E1 = 1.1 <0, E2 = 1<0
I1 = 1.1 / (1j*z[0][3])
I2 = 1 / (1j*z[1][3])



##  Building Admittance Matrix (Ybus)


In [ ]:
# current injection vector (size must match the number of buses)
# 0 is ground node, so we exclude it from the admittance matrix
# buses 1..max_bus are the actual buses, so total buses = max_bus
number_busses = int(np.max(z[:, :2]))  # 4 buses (1, 2, 3, 4)

Ibus = np.zeros(number_busses, dtype=complex)
Ibus[0] = I1  # bus 1
Ibus[1] = I2  # bus 2

# building the admittance matrix (excluding ground node 0)
def admittance_matrix(zdata, nbuses=None):
    zdata = np.asarray(zdata)

    from_data = zdata[:, 0].astype(int)
    to_data   = zdata[:, 1].astype(int)
    r_data    = zdata[:, 2].astype(float)
    x_data    = zdata[:, 3].astype(float)

    if nbuses is None:
        nbuses = int(max(from_data.max(), to_data.max()))

    Y = np.zeros((nbuses, nbuses), dtype=complex)

    for k in range(zdata.shape[0]):
        i = from_data[k]
        j = to_data[k]
        y = 1.0 / (r_data[k] + 1j * x_data[k])

        # If connected to ground (bus 0), only add to diagonal
        if i == 0:
            Y[j-1, j-1] += y  # shift index by 1 since we exclude ground
        elif j == 0:
            Y[i-1, i-1] += y
        else:
            # Both buses are non-ground, shift indices by 1
            Y[i-1, i-1] += y
            Y[j-1, j-1] += y
            Y[i-1, j-1] -= y
            Y[j-1, i-1] -= y

    return Y

Ybus = admittance_matrix(z, number_busses)
print("Ybus =\n",Ybus)

# Ibus = Ybus * Vbus => Vbus = inv(Ybus) * Ibus
invYbus = np.linalg.inv(Ybus)
print("Inv(Ybus) = \n",invYbus)


vbus = np.matmul(invYbus, Ibus)
print("Vbus =\n",vbus)

0
1
2
3
4
5
Ybus =
 [[0. -8.5j  0. +2.5j  0. +5.j   0. +0.j  ]
 [0. +2.5j  0. -8.75j 0. +5.j   0. +0.j  ]
 [0. +5.j   0. +5.j   0.-22.5j  0.+12.5j ]
 [0. +0.j   0. +0.j   0.+12.5j  0.-12.5j ]]
Inv(Ybus) = 
 [[0.+0.5j   0.+0.4j   0.+0.45j  0.+0.45j ]
 [0.+0.4j   0.+0.48j  0.+0.44j  0.+0.44j ]
 [0.+0.45j  0.+0.44j  0.+0.545j 0.+0.545j]
 [0.+0.45j  0.+0.44j  0.+0.545j 0.+0.625j]]
Vbus =
 [1.05 +0.j 1.04 +0.j 1.045+0.j 1.045+0.j]
